In [0]:
%sql
-- 02_transformation_for_analytics
-- transform data ingested for analytical purposes

-- SELECT
--     *
-- FROM
--     workspace.default.stock_data;

CREATE OR REPLACE VIEW workspace.default.stock_metrics AS
WITH BASE_DATA AS (
SELECT
    TICKER,
    DATETIME,
    CLOSE,
    IFNULL(LAG(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME),CLOSE) AS PREV_CLOSE,
    (CLOSE-PREV_CLOSE)/PREV_CLOSE AS DAILY_RETURN,
    (CLOSE - FIRST_VALUE(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME)) / FIRST_VALUE(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME) AS CUM_RETURN,
    HIGH - LOW AS DAILY_RANGE,
    (HIGH - LOW) / OPEN AS DAILY_VOLATILITY,
    VOLUME / AVG(VOLUME) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) AS VOLUME_SPIKE,
    AVG(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 15 PRECEDING AND CURRENT ROW) AS MA_15,
    AVG(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) AS MA_30,
    AVG(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 60 PRECEDING AND CURRENT ROW) AS MA_60
FROM
    (SELECT
        TICKER,
        DATETIME::DATE,
        OPEN,
        HIGH,
        LOW,
        CLOSE,
        VOLUME
    FROM
        workspace.default.stock_data
    QUALIFY ROW_NUMBER() OVER (PARTITION BY TICKER, DATETIME ORDER BY UPDATED_ON)=1)
ORDER BY
    DATETIME ASC,
    TICKER
)
SELECT
    *,
    MA_15 - 2*STDDEV(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 15 PRECEDING AND CURRENT ROW) AS MA_15_LOWER_BB,
    MA_30 - 2*STDDEV(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) AS MA_30_LOWER_BB,
    MA_60 - 2*STDDEV(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 60 PRECEDING AND CURRENT ROW) AS MA_60_LOWER_BB,
    MA_15 + 2*STDDEV(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 15 PRECEDING AND CURRENT ROW) AS MA_15_UPPER_BB,
    MA_30 + 2*STDDEV(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) AS MA_30_UPPER_BB,
    MA_60 + 2*STDDEV(CLOSE) OVER (PARTITION BY TICKER ORDER BY DATETIME ROWS BETWEEN 60 PRECEDING AND CURRENT ROW) AS MA_60_UPPER_BB
FROM
    BASE_DATA;



In [0]:
%sql
SELECT
    *
FROM
    workspace.default.stock_metrics;